In [1]:
import os
import glob
import torch
import random
import numpy as np

from cuml.preprocessing.LabelEncoder import LabelEncoder

import cupy as cp
import cudf as cd

import ray
from ray import tune, train as raytrain
from ray.train                    import Checkpoint
from ray.tune.schedulers          import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback
from ray.tune.search.hyperopt     import HyperOptSearch

import xgboost as xgb

# Set Seed For Reproducibillity

In [2]:
def set_reproducibillity(seed=None):
    if seed is None:
        seed = 3407
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    cp.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed
seed = set_reproducibillity(8120)

# preprocessisng data

In [3]:
LOG_PATH = os.path.join(os.path.abspath(os.getcwd()), "ray_results")
train_url = "../data/titanic/train.csv"
test_url  = "../data/titanic/test_augmented.csv"
features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Embarked_l", "Fare"]


train = cd.read_csv(train_url)
test  = cd.read_csv(test_url)

train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)
train['Fare'].fillna(train['Fare'].mean(), inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

le = LabelEncoder()
train["Embarked_l"] = le.fit_transform(train.Embarked)
test["Embarked_l"]  = le.fit_transform(test.Embarked)

train_y = train['Survived']
train_x = cd.get_dummies(train[features]).astype(cp.float32)
test_y  = test['Survived']
test_x  = cd.get_dummies(test[features]).astype(cp.float32)
model = None

# XGBoost settings

In [4]:
dtest  = xgb.DMatrix(test_x, test_y)
dtrain = xgb.DMatrix(train_x, train_y)

# Default XGBoost

In [5]:
config = {
     'verbosity': 2,
    'objective':  'binary:logistic',
    'num_class': 1,
    'grow_policy': 'lossguide',
    'learning_rate': 0.001,
    'max_leaves': 64,
    'max_depth': 6,
    'eval_metric':  'auc',
    'tree_method': 'hist',
    'device':"cuda",
    'min_child_weight': 7,
    'colsample_bytree': 0.5,
    'lambda': 0.5,
    'alpha': 1, 
    'seed' : seed
}

model   = xgb.train(config, dtrain, num_boost_round=100, evals=[[dtest, 'train']], verbose_eval=False)

# Predict For New Params
predict = model.predict(dtest)
print(cp.mean(cp.array(predict > 0.5).astype(cp.float32) == cp.array(test_y)).item())
del model

[13:52:34] INFO: /home/conda/feedstock_root/build_artifacts/xgboost-split_1732221635527/work/src/data/simple_dmatrix.cc:139: Generating new Ellpack page.
0.6220095693779905


In [6]:
def train_price(config):
    # This is a simple training function to be passed into Tune
    # Split into train and test set    
    dtest  = xgb.DMatrix(test_x, test_y)
    dtrain = xgb.DMatrix(train_x, train_y)
    model   = xgb.train(config, dtrain, num_boost_round=100, evals=[[dtest, 'train']], verbose_eval=False)

    # Predict For New Params
    predict = model.predict(dtest)
    mean = cp.asnumpy(cp.mean(cp.array(predict > 0.5).astype(cp.float32) == cp.array(test_y)))
    raytrain.report(metrics={"mean_accuracy": float(mean)})#, checkpoint=Checkpoint.from_directory(LOG_PATH))
    
def tune_xgboost():
    search_space = {
        # You can mix constants with search space objects.
        "objective"       : 'binary:logistic',
        "eval_metric"     : 'auc',
        'learning_rate'   : tune.loguniform(1e-5, 1e-2),
        'max_leaves'      : tune.lograndint(16,128),
        "max_depth"       : tune.lograndint(1,10),
        "min_child_weight": tune.uniform(1,10),
        'colsample_bytree': tune.uniform(0.5, 1.0),
        'lambda'          : tune.uniform(0, 10),
        'alpha'           : tune.uniform(0, 1),
        'grow_policy': 'lossguide',
        'tree_method': 'hist',
        'device':"cuda",
        'num_class': 1,
        'seed' : seed
    }
    # This will enable aggressive early stopping of bad trials.
    scheduler = ASHAScheduler(
        max_t=100,  # 100 training iterations
        grace_period=4,
        reduction_factor=2)
    tpe_search =  HyperOptSearch(metric="mean_accuracy", mode="max", random_state_seed=seed)
    
    analysis = tune.run(
        train_price,
        metric="mean_accuracy",
        mode="max",
        search_alg=tpe_search,
        # You can add "gpu": 0.1 to allocate GPUs
        resources_per_trial={"cpu": 2, "gpu": 0.1},
        config=search_space,
        num_samples=80,
        storage_path=LOG_PATH,
        scheduler=scheduler)

    return analysis

# Init RAY and Run

In [7]:
ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
ray.init(include_dashboard=True, log_to_driver=False)

2025-04-30 13:52:36,288	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.16
Ray version:,2.40.0
Dashboard:,http://127.0.0.1:8265


In [8]:
Checkpoint.from_directory('.')

Checkpoint(filesystem=local, path=.)

In [9]:
analysis = tune_xgboost()

2025-04-30 13:52:37,024	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,mean_accuracy
train_price_02dac4bf,0.62201
train_price_038c84a0,0.770335
train_price_07b95e33,0.62201
train_price_0d0fb185,0.62201
train_price_13c04194,0.779904
train_price_14d6688d,0.667464
train_price_158ef4db,0.62201
train_price_15c24c1e,0.62201
train_price_1a6e2715,0.77512
train_price_1d80d0fb,0.77512


2025-04-30 13:55:54,011	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/media/HDD/JeYoung/Fundamentals_of_GPU_DataScience_with_RAPIDS/Chapter07/ray_results/train_price_2025-04-30_13-52-37' in 0.0397s.
2025-04-30 13:55:54,039	INFO tune.py:1041 -- Total run time: 197.02 seconds (196.92 seconds for the tuning loop).


In [10]:
from ray.tune import ExperimentAnalysis

analysis = ExperimentAnalysis(glob.glob(os.path.join(LOG_PATH,"*"))[-1], default_metric="mean_accuracy", default_mode="max")
analysis.best_result_df

,mean_accuracy,timestamp,checkpoint_dir_name,done,training_iteration,date,time_this_iter_s,time_total_s,pid,hostname,...,config/max_depth,config/min_child_weight,config/colsample_bytree,config/lambda,config/alpha,config/grow_policy,config/tree_method,config/device,config/num_class,config/seed
trial_id,,,,,,,,,,,,,,,,,,,,,
7fa207eb,0.779904,1745988933,None,True,1,2025-04-30_13-55-33,0.471396,0.471396,799604,cvmi-jeyoung-System-Product-Name,...,5,3.629607,0.945509,3.380119,0.649328,lossguide,hist,cuda,1,8120
